In [1]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from empyrical import max_drawdown, alpha, beta
from sklearn.preprocessing import MinMaxScaler
from src.twitter_interface import TwitterInterface
from src.asset_selector import AssetSelector
from src.indicators import Indicators
from util import time_formatter
from src.predictor import Predictor
from sklearn.cluster import KMeans
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as py
import pandas as pd
import numpy as np
import configparser
import requests
import twitter
import json
import time
import sys
import os

weeks = 13

backdate = time_formatter(time.time() - (604800 * weeks))
config = configparser.ConfigParser()
try:
    config.read(os.path.relpath("config.ini"))
except FileExistsError as e:
    print("FileExistsError: {}".format(e))
    sys.exit(1)
alpaca_api = tradeapi.REST(
    base_url    = config["alpaca"]["APCA_API_BASE_URL"],
    key_id      = config["alpaca"]["APCA_API_KEY_ID"],
    secret_key  = config["alpaca"]["APCA_API_SECRET_KEY"],
    api_version = config["alpaca"]["VERSION"]
)
twitter_api = twitter.Api(
    config["twitter"]["CONSUMER_KEY"],
    config["twitter"]["CONSUMER_SECRET"],
    config["twitter"]["ACCESS_TOKEN_KEY"],
    config["twitter"]["ACCESS_TOKEN_SECRET"]
)
tq_key = config["tenquant"]["api_key"]

tdf = pd.DataFrame()
ti = TwitterInterface(twitter_api, tdf)
trading_account = alpaca_api.get_account()

selector = AssetSelector(alpaca_api, edgar_token=None)

def get_sentiment(text):
    """Given a text block, return a sentiment score based.
    :param text:
    :return:
    """
    text_polarity   = sid.polarity_scores(text)

    if text_polarity["compound"] > 0.50:
        sentiment   = "positive"
    else:
        sentiment   = "negative"
    return sentiment, text_polarity["compound"]

def get_losers():
    print('Losers'.center(45))
    print()
    print('Ticker'.ljust(10), 'Last'.ljust(11), 'Change'.ljust(11), 'Pct Loss')
    print("{:<30}".format('–' * 45))
    for symbol in range(len(alpaca_api.polygon.gainers_losers("losers"))):
        
        # print out some indicators here # TODO
        # also throw out shitty stocks
        
        print(alpaca_api.polygon.gainers_losers("losers")[symbol].ticker.ljust(10),
              '$' + str(alpaca_api.polygon.gainers_losers("losers")[symbol].lastTrade['p']).ljust(10),
              '$' + str(alpaca_api.polygon.gainers_losers("losers")[symbol].todaysChange).ljust(10),
              str(alpaca_api.polygon.gainers_losers("losers")[symbol].todaysChangePerc)+'%')

def get_gainers():
    print('Gainers'.center(45))
    print()
    print('Ticker'.ljust(10), 'Last'.ljust(11), 'Change'.ljust(11), 'Pct Gain')
    print("{:<30}".format('–' * 45))
    for symbol in range(len(alpaca_api.polygon.gainers_losers())):
        
        # print out some indicators here # TODO
        # also throw out shitty stocks
        
        print(alpaca_api.polygon.gainers_losers()[symbol].ticker.ljust(10),
              '$' + str(alpaca_api.polygon.gainers_losers()[symbol].lastTrade['p']).ljust(10),
              '$' + str(alpaca_api.polygon.gainers_losers()[symbol].todaysChange).ljust(10),
              str(alpaca_api.polygon.gainers_losers()[symbol].todaysChangePerc)+'%')

/home/ben/anaconda3/envs/alpaca_test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ben/anaconda3/envs/alpaca_test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ben/anaconda3/envs/alpaca_test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ben/anaconda3/envs/alpaca_test/lib

In [2]:
if __name__ == '__main__':
    get_gainers()
    print()
    get_losers()
    print()
    print("Bulls".center(45))
    bulls = selector.bullish_candlesticks(64, 20)
    print()
    print("Bears".center(45))
    bears = selector.bearish_candlesticks(64, 20)

                   Gainers                   

Ticker     Last        Change      Pct Gain
–––––––––––––––––––––––––––––––––––––––––––––
ISR        $0.452      $0.111      264.314%
ISR        $2.3        $0.46       25%
SINT       $2.3        $0.2        19.608%
  0         $2583852325273.6 $0.2        17.925%
YTEN       $0.38       $0.05       15.152%
CHK        $0.765      $0.095      14.213%
FCEL       $25.8       $3.11       13.706%
CHK        $0.76       $0.09       13.467%
AXU        $1.95       $0.22       12.717%
DDOG       $38.77      $4.37       12.703%
PTI        $1.2        $0.13       12.15%
KNSL       $97         $8.02       9.013%
ORMP       $2.75       $0.21       8.268%
ADPT       $30.15      $1.8        6.349%
GOOS       $41.42      $2.38       6.096%
BLRX       $2.62       $0.13       5.221%
LODE       $0.094      $0.005      5.145%
ARD        $20         $0.95       4.987%
YANG       $50.02      $2.29       4.798%
SCOR       $3.39       $0.15       4.63%
NBRV       

In [3]:
ticker = "ZVZZT"

In [4]:
sid = SentimentIntensityAnalyzer()
stocktwits = requests.post("https://api.stocktwits.com/api/2/streams/symbol/{}.json".format(ticker))
stocktwits = json.loads(stocktwits.text)
stocktwitstext = "\n".join([m["body"] for m in stocktwits["messages"]])
tweets = ti.get_tweets(ticker, "stock")

KeyError: 'messages'

In [5]:
stsentiment, stscore = get_sentiment(stocktwitstext)
tsentiment, tscore = get_sentiment(tweets)

NameError: name 'stocktwitstext' is not defined

In [6]:
print("Sentiment score for {}".format(ticker))
print("{:<30}".format('–' * 45))
print("Stocktwits:\t{}\t score: {}\nTwitter:\t{}\t score:\t{}".format(stsentiment, stscore, tsentiment, tscore))
print("Average:\t{}".format((stscore + tscore)/2))

Sentiment score for ZVZZT
–––––––––––––––––––––––––––––––––––––––––––––


NameError: name 'stsentiment' is not defined

In [7]:
# alpaca_api.polygon.historic_trades(ticker, '2019-10-10', limit=10)
# alpaca_api.polygon.news(ticker)
# alpaca_api.polygon.last_trade(ticker)
# alpaca_api.polygon.last_quote(ticker)
# alpaca_api.polygon.company(ticker)
# alpaca_api.polygon.dividends(ticker)
# alpaca_api.polygon.financials(ticker)

In [8]:
tenquant_data = requests.get("https://api.tenquant.io/data?key={}&ticker={}".format(tq_key, ticker))
tenquant_data = json.loads(tenquant_data.text)

In [9]:
stocktwits

{'response': {'status': 200},
 'symbol': {'id': 14368,
  'symbol': 'OSS',
  'title': 'One Stop Systems',
  'aliases': [],
  'is_following': False,
  'watchlist_count': 787},
 'cursor': {'more': True, 'since': 183216956, 'max': 182942498},
 'messages': [{'id': 183216956,
   'body': '$RIGL $NEOS $OSS can y’all hold &amp; be bullish enough to produce a continuation? \n\n🦆🐂',
   'created_at': '2019-11-11T04:40:22Z',
   'user': {'id': 1993653,
    'username': 'Lynssss3125',
    'name': 'Lyns',
    'avatar_url': 'https://avatars.stocktwits.com/production/1993653/thumb-1556827112.png',
    'avatar_url_ssl': 'https://avatars.stocktwits.com/production/1993653/thumb-1556827112.png',
    'join_date': '2019-04-07',
    'official': False,
    'identity': 'User',
    'classification': [],
    'followers': 108,
    'following': 16,
    'ideas': 185,
    'watchlist_stocks_count': 18,
    'like_count': 94,
    'plus_tier': '',
    'premium_room': ''},
   'source': {'id': 1149,
    'title': 'StockTwits 

In [10]:
tweets 

'@TriatletasenRed @MediaMarkt_es Hola, si te refieres a este modelo, actualmente disponemos de stock. Un saludo https://t.co/ZkFaachgUf\n\nThe satchel or the male handbag. Although the backpack is arguably lindy for carrying lots of stuff. You can buy a… https://t.co/kfc9R9aQLC\n\n#Earnings Today-P4\n$ONTO $OSS $PCTI $PEGA $PEN $PETQ $PFNX $PLNT $PRAA $PROF $PVAC $QNST $RBA $RDVT $RGR $RLH $RLJ… https://t.co/HCkXI5UiKf\n\n#Earnings Tomorrow-P4\n$ONTO $OSS $PCTI $PEGA $PEN $PETQ $PFNX $PLNT $PRAA $PROF $PVAC $QNST $RBA $RDVT $RGR $RLH… https://t.co/WTvtTcLK1r\n\nVi är börja närma oss trendlinjen nu $FING ”stock market is a device for transferring money from the impatient to the patient”\n\n@Vahastu Kan vi förvänta oss 8-10 år uppgång nu när vi har haft en börskrasch?'

In [11]:
tenquant_data

{'assets': 37595493.0,
 'bookvalue': 27447056.0,
 'comprehensiveincome': -2060843.0,
 'comprehensiveincomeattributabletononcontrollinginterest': 0.0,
 'comprehensiveincomeattributabletoparent': -2060843.0,
 'costofrevenue': 27028399.0,
 'currencycode': 'USD',
 'currentassets': 22482611.0,
 'currentliabilities': 9561056.0,
 'date': '2019-09-30',
 'dividendpayments': 0.0,
 'dividendyield': 0.0,
 'documenttype': '10-Q',
 'duration': 3,
 'equity': 27447056.0,
 'exchangegainslosses': 0.0,
 'extraordaryitemsgainloss': 0.0,
 'grossprofit': 12854700.0,
 'incomebeforeequitymethodinvestments': -1994461.0,
 'incomefromcontinuingoperationsaftertax': -1994461.0,
 'incomefromcontinuingoperationsbeforetax': -2589351.0,
 'incomefromequitymethodinvestments': 0.0,
 'incometaxexpensebenefit': -594890.0,
 'interestanddebtexpense': 111463.0,
 'liabilities': 10148437.0,
 'liabilitiesandequity': 37595493.0,
 'marketcap': 40896540,
 'netcashflow': 1987366.0,
 'netcashflowsfinancing': 3659731.0,
 'netcashflows